In [5]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import re
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report



In [6]:
# Prendre en main les données
data = pd.read_csv("../data/train.csv")



In [7]:
data.columns


Index(['doc_id', 'titre', 'type', 'difficulte', 'cout', 'ingredients',
       'recette'],
      dtype='object')

In [8]:
#methode Baseline
X = data.drop('type', axis=1)
y = data['type']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 42)

dummy = DummyClassifier(strategy='stratified')
dummy.fit(X_train, y_train)

y_pred = dummy.predict(X_test)

print(classification_report(y_test, y_pred))


                precision    recall  f1-score   support

       Dessert       0.27      0.29      0.28       726
        Entrée       0.24      0.23      0.24       611
Plat principal       0.45      0.44      0.45      1158

      accuracy                           0.35      2495
     macro avg       0.32      0.32      0.32      2495
  weighted avg       0.35      0.35      0.35      2495



In [9]:
### Run2: NOMMETHODE
### Run3: NOMMETHODE
### Run4: NOMMETHODE (pour aller plus loin)

train_reviews = X_train
test_reviews = X_test
train_reviews

,doc_id,titre,difficulte,cout,ingredients,recette
8260,recette_170277.xml,Langue de boeuf sauce piquante aux cornichons,Facile,Moyen,"- 1 langue de 1,5 kg - 20 cl de vinaigre d'alc...",Faire tremper la langue toute une nuit dans un...
9833,recette_21519.xml,Tarte aux raisins et amandes,Facile,Moyen,- 1 pâte brisée - 150 g de raisins bien mûrs -...,Garnissez une tourtière avec la pâte étalée. F...
1454,recette_72746.xml,Meringues bien sèches,Très facile,Bon marché,- 2 blancs d'oeufs - 60 g de sucre en poudre -...,"Préchauffez le four à 150°, thermostat 5. Ajou..."
4954,recette_38858.xml,"Sandwich jambon, menthe et feta",Très facile,Moyen,- 3 tranches de pain complet ou de seigle ou d...,Faire la petit sauce à la menthe : mêler crème...
6369,recette_167836.xml,Salade de mâche et lardons,Très facile,Bon marché,- 300 g de mâche - quelques feuilles de roquet...,Faire revenir les lardons et les déposer chaud...
...,...,...,...,...,...,...
11964,recette_66759.xml,"Cake à l'orange, poisson et carottes",Très facile,Bon marché,- 2 filets de poisson cuit - 1 orange - 2 bell...,Préchauffer le four à 180°C (thermostat 6).Cou...
5191,recette_52749.xml,Bananes au citron,Très facile,Bon marché,- 4 bananes - 1 sachet de sucre vanillé - 1/2 ...,Couper les bananes en rondelles et les mettre ...
5390,recette_32633.xml,Moussaka au thon,Facile,Bon marché,- 600 g de thon au naturel (boîte) - 3 auberg...,Laver et trancher fin les aubergines dans la l...
860,recette_64624.xml,Lasagnes à la ratatouille et au fromage,Très facile,Bon marché,- 500 g de lasagnes - 6 tranches de jambon cui...,Emincer le demi-oignon. Peler les tomates et l...


In [13]:

stopwords = nltk.corpus.stopwords.words('french')

def clean_text(text):
    text = re.sub(r'<.*?>', '', text) #Remove HTML tags
    text = re.sub(r'[^a-zA-Z]', ' ', text) #Remove punctuation
    text = text.lower() #Convert to lowercase
    text = text.split() #Split the text
    text = [word for word in text if not word in set(stopwords)] #Remove stopwords
    text = ' '.join(text)
    return text

def stemmer(text):
    text = text.split()
    stemmer = nltk.PorterStemmer()
    stemmed_words = [stemmer.stem(word) for word in text]
    text = ' '.join(stemmed_words)
    return text

In [14]:


# Lisez vos données
data = pd.read_csv("../data/train.csv")

data['combined'] = data['titre'] + ' ' + data['difficulte'] + ' ' + data['cout'].astype(str) + ' ' + data['ingredients'] + ' ' + data['recette']
data['combined'] = data['combined'].apply(clean_text)
data['combined'] = data['combined'].apply(stemmer)

X = data['combined']
y = data['type']

# Divisez vos données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

tfidf = TfidfVectorizer()

X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

svm = SVC(kernel='linear', C=1, random_state=42)
svm.fit(X_train_tfidf, y_train)

y_pred = svm.predict(X_test_tfidf)

print(classification_report(y_test, y_pred))

                precision    recall  f1-score   support

       Dessert       0.99      1.00      0.99       726
        Entrée       0.77      0.66      0.71       611
Plat principal       0.84      0.89      0.86      1158

      accuracy                           0.87      2495
     macro avg       0.86      0.85      0.86      2495
  weighted avg       0.86      0.87      0.86      2495

